In [17]:
import sys

sys.path.append("src")

In [36]:
processing_year = "2023-24"

# debits to ignore always
ignore_debits = [
  "BANAYANTREESERVICES",
  "Nippon India"
]

In [19]:
import get_data
import pandas as pd
import logging

logger = logging.getLogger(__name__)

data = get_data.load_statement(processing_year)

In [20]:
statement = data

# this was to evaluate what an empty value looks like
# empty_value = statement["DR"][975]
# logger.info(type(empty_value))
# logger.info(len(empty_value))
# logger.info(empty_value == " ") # <- returned true

statement["CR"] = statement["CR"].str.strip()
statement["DR"] = statement["DR"].str.strip()

statement["DR"] = statement["DR"].replace(to_replace="",value="0.00")
statement["CR"] = statement["CR"].replace(to_replace="",value="0.00")

statement["DR"] = statement["DR"].astype(float).astype(int)
statement["CR"] = statement["CR"].astype(float).astype(int)

credit_records = statement[statement["DR"] == 0]
debit_records  = statement[statement["CR"] == 0]

# logger.info(credit_records)
# logger.info(debit_records)

total_credits = credit_records["CR"].sum()
total_debits  = debit_records["DR"].sum()

logger.info(f"Total Credits: {total_credits}")
logger.info(f"Total Debits: {total_debits}")

# credits which have USD in their PARTICULARS column
usd_credits = credit_records[credit_records["PARTICULARS"].str.contains("USD")]
usd_credits_total = usd_credits["CR"].sum()

# Print each USD credit record with Tran Date, BAL and SOL
for index, row in usd_credits.iterrows():
  print(f"{index}: {row["Tran Date"]}\t{row["PARTICULARS"]}\t{row["CR"]}\t{row["BAL"]}\t{row["SOL"]}")

logger.info(f"USD Credits: {usd_credits_total}")

2024-09-20 23:02:58 [INFO] 3423774088.py:27 - Total Credits: 5746514
2024-09-20 23:02:58 [INFO] 3423774088.py:28 - Total Debits: 5334731
2024-09-20 23:02:58 [INFO] 3423774088.py:38 - USD Credits: 4704911


12: 06-04-2023	BRN-REF NO.0019RIR2300399 USD 3960@80.1425/RLZ	316693	1867939.15	19
86: 18-05-2023	BRN-REF NO.0019RIR2300563 USD 4048@80.5350/RLZ	325504	1875737.63	19
147: 15-06-2023	BRN-REF NO.0019RIR2300676 USD 5346@80.5025/RLZ	429771	2161960.85	19
199: 10-07-2023	BRN-REF NO.0019RIR2300766 USD 5292@80.9450/RLZ	427767	2254184.41	19
279: 08-08-2023	BRN-REF NO.0019RIR2300886 USD 5346@81.1100/RLZ	433015	2225247.76	19
389: 28-09-2023	BRN-REF NO.0019RIR2301105 USD 5508@81.5425/RLZ	448523	2517443.68	19
496: 01-11-2023	BRN-REF NO.0019RIR2301260 USD 5454@81.5725/RLZ	444288	2679432.28	19
656: 04-12-2023	BRN-REF NO.0019RIR2301387 USD 5238@81.5900/RLZ	426775	2652725.0	19
768: 16-01-2024	BRN-REF NO.0019RIR2400058 USD 7722@81.4850/RLZ	628452	2779243.42	19
851: 13-02-2024	BRN-REF NO.0019RIR2400155 USD 4968@81.3475/RLZ	403562	1998392.9	19
930: 11-03-2024	BRN-REF NO.0019RIR2400249 USD 5184@81.2400/RLZ	420561	2083555.54	19


In [41]:
# Netherlands Costs

# Get all records which have a Tran Date between 2023-11-01 and 2023-11-16
statement["Tran Date"] = pd.to_datetime(statement["Tran Date"],format="%d-%m-%Y")

netherlands_costs = statement[(statement["Tran Date"] >= "2023-11-01") & (statement["Tran Date"] <= "2023-11-16")]

# total debits for netherlands costs
netherlands_debits  = netherlands_costs[netherlands_costs["DR"] > 0]
netherlands_credits = netherlands_costs[netherlands_costs["CR"] > 0]

# find debits which are to be ignored
def ignore_debits_fn(row):
  for debit in ignore_debits:
    particular_str : str = row["PARTICULARS"]
    if particular_str.find(debit) != -1:
      return True
  return False

ignored_debits = netherlands_debits.apply(ignore_debits_fn, axis=1)



,Tran Date,CHQNO,PARTICULARS,DR,CR,BAL,SOL
1,02-04-2023,-,ACH-DR-BANAYANTREESERVICESL-ETMONEYXXXLYFjXA1qAdm,5035,0,1594235.08,454
4,03-04-2023,-,Nippon India Sma/87482322/SCGP,3000,0,1567749.08,454
54,01-05-2023,-,ACH-DR-BANAYANTREESERVICESL-ETMONEYXXXLk8rha6ZNAL,5035,0,1609587.64,454
59,02-05-2023,-,Nippon India Sma/88804730/SCGP,3000,0,1603377.23,454
118,31-05-2023,-,Nippon India Sma/90157623/SCGP,3000,0,1770765.41,454
119,01-06-2023,-,ACH-DR-BANAYANTREESERVICESL-ETMONEYXXXLwPP5LsatHK,5035,0,1765730.01,454
178,01-07-2023,-,ACH-DR-BANAYANTREESERVICESL-ETMONEYXXXM8HOx5XtJKb,5035,0,1843571.20,454
183,03-07-2023,-,Nippon India Sma/91633897/SCGP,3000,0,1853474.20,454
238,01-08-2023,-,Nippon India Sma/93043372/SCGP,3000,0,1936207.48,454
239,01-08-2023,-,ACH-DR-BANAYANTREESERVICESL-ETMONEYXXXMKY2sERF511,5035,0,1931172.08,454
